# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [30]:
# import libraries
import re
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator, TransformerMixin

import pickle

[nltk_data] Downloading package punkt to /Users/michael/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/michael/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/michael/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse.db', engine)

In [3]:
# replace the value 2 of columns related to 0
# as they are not disaster messages
# (ASK A MENTOR: https://knowledge.udacity.com/questions/954962)
df.related = df.related.replace(2, 0)

In [4]:
X = df.message.values
Y = df.iloc[:, 4:].values

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # stop_words = stopwords.words("english") + list(string.punctuation)
    # lemmatizer = WordNetLemmatizer()
    
    # text = text.lower().strip()
    # tokens = word_tokenize(text)
    
    # tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # return tokens

    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(
        class_weight='balanced',
        n_jobs = -1
        ))),])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
pipeline.fit(X_train, Y_train)

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f9b894f8ca0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        n_jobs=-1)))])

In [8]:
pipeline.score(X_train, Y_train)

0.9975078832265283

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
def model_score (model):

    Y_pred = model.predict(X_test)

    list_precision, list_recall, list_f1, list_accuracy = [], [], [], []

    # Iterate 36 target columns and generate a classification report for each
    for i, col in enumerate(df.columns[4:]):
        
        rslt = classification_report(Y_test[:, i], Y_pred[:, i])
        
        # weighted avg scores are in the 2nd last line: 
        score_line = rslt.split('\n')[-2]
        score_line_split = score_line.split()
        
        # scores are in the 2nd to 4th places of splitted texts of score_line
        precision_score = float (score_line_split[2])
        list_precision.append(precision_score)
        
        recall_score = float (score_line_split[3])
        list_recall.append(recall_score)
        
        f1_score = float (score_line_split[4])
        list_f1.append(f1_score)
        
        # accuracy is the same as weighted avg recall
        # accuracy_score = float (rslt.split('\n')[-4].split()[1])
        # list_accuracy.append(accuracy_score)
        
        print(f'{i} Target column {col}:')
        print(rslt)
        print()
        
    print ('mean of weighted avg precision: {:.2f}'.format(sum(list_precision)/len(list_precision)))
    print ('mean of weighted avg recall: {:.2f}'.format(sum(list_recall)/len(list_recall)))
    print ('mean of weighted avg f1: {:.2f}'.format(sum(list_f1)/len(list_f1)))
    print ('mean of accuracy: {:.2f}'.format(sum(list_recall)/len(list_recall)))

In [10]:
model_score (pipeline)

0 Target column related:
              precision    recall  f1-score   support

           0       0.72      0.37      0.49      1561
           1       0.83      0.95      0.89      4993

    accuracy                           0.82      6554
   macro avg       0.77      0.66      0.69      6554
weighted avg       0.80      0.82      0.79      6554


1 Target column request:
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5400
           1       0.89      0.42      0.57      1154

    accuracy                           0.89      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.89      0.87      6554


2 Target column offer:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/p

In [11]:
pipeline.score(X_test, Y_test)

0.23924321025328044

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f9b894f8ca0>)),
  ('tfidf', TfidfTransformer()),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                          n_jobs=-1)))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f9b894f8ca0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                        n_jobs=-1)),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b

In [13]:
parameters = {
    'clf__estimator__n_estimators': [100, 300, 500],
    'clf__estimator__min_samples_split': [2, 6, 10, 14, 18],
    }

# Create a grid search object using the pipeline and parameters
grid_search = GridSearchCV(pipeline, parameters, cv = 3, verbose = 3)
# grid_search = GridSearchCV(pipeline, parameters, cv = 3, verbose=3, scoring = 'accuracy')

# Fit the grid search to the data
grid_search.fit(X_train, Y_train)

# Print the best hyperparameters found by grid search
print("Best hyperparameters:", grid_search.best_params_)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.233 total time=  37.2s


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.230 total time=  38.3s


...

[CV 2/3] END clf__estimator__min_samples_split=18, clf__estimator__n_estimators=500;, score=0.264 total time= 2.1min


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/3] END clf__estimator__min_samples_split=18, clf__estimator__n_estimators=500;, score=0.271 total time= 2.1min


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best hyperparameters: {'clf__estimator__min_samples_split': 18, 'clf__estimator__n_estimators': 500}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
model_score (grid_search)

0 Target column related:
              precision    recall  f1-score   support

           0       0.68      0.49      0.57      1561
           1       0.85      0.93      0.89      4993

    accuracy                           0.82      6554
   macro avg       0.77      0.71      0.73      6554
weighted avg       0.81      0.82      0.81      6554


1 Target column request:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      5400
           1       0.76      0.61      0.68      1154

    accuracy                           0.90      6554
   macro avg       0.84      0.79      0.81      6554
weighted avg       0.89      0.90      0.89      6554


2 Target column offer:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/py/lib/p

In [15]:
grid_search.score(X_test, Y_test)

0.27021666158071406

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
# Improve tokenize by adding stop words
def tokenize_improved(text):
    
    # add stop_words
    stop_words = stopwords.words("english") + list(string.punctuation)
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        if tok not in stop_words:       
            # lemmatize, normalize case, and remove leading/trailing white space
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)
    return clean_tokens

In [24]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [25]:
pipeline_improved = Pipeline([
    ('features', FeatureUnion(
        [('text_pipeline',
          Pipeline([
              ('vect', CountVectorizer(tokenizer=tokenize_improved)),
              ('tfidf', TfidfTransformer()),
              ])),
         ('isverb', StartingVerbExtractor())
         ])),
    ('clf', MultiOutputClassifier(
        RandomForestClassifier(
        class_weight='balanced',
        n_jobs = -1
        )))
    ])




# pipeline_improved = Pipeline([
#     ('vect', CountVectorizer(tokenizer=tokenize_improved)),
#     ('tfidf', TfidfTransformer()),
#     ('clf', MultiOutputClassifier(RandomForestClassifier(
#         class_weight='balanced',
#         n_jobs = -1
#         ))),])

In [26]:
pipeline_improved.fit(X_train, Y_train)

/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize_improved at 0x7f9b6bb0f0a0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('isverb',
                                                 StartingVerbExtractor())])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        n_jobs=-1)))])

In [27]:
pipeline_improved.score(X_test, Y_test)

0.2740311260299054

In [28]:
parameters = {
    'clf__estimator__n_estimators': [100, 300, 500],
    'clf__estimator__min_samples_split': [2, 6, 10, 14, 18],
    }

# Create a grid search object using the pipeline and parameters
grid_search_improved = GridSearchCV(pipeline_improved, parameters, cv = 3, verbose = 3)
# grid_search = GridSearchCV(pipeline, parameters, cv = 3, verbose=3, scoring = 'accuracy')

# Fit the grid search to the data
grid_search_improved.fit(X_train, Y_train)

# Print the best hyperparameters found by grid search
print("Best hyperparameters_improved:", grid_search.best_params_)

Fitting 3 folds for each of 15 candidates, totalling 45 fits


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.259 total time=  56.6s


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100;, score=0.260 total time=  55.1s


...

[CV 2/3] END clf__estimator__min_samples_split=18, clf__estimator__n_estimators=500;, score=0.276 total time= 2.2min


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/3] END clf__estimator__min_samples_split=18, clf__estimator__n_estimators=500;, score=0.287 total time= 2.3min


/opt/anaconda3/envs/py/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best hyperparameters_improved: {'clf__estimator__min_samples_split': 18, 'clf__estimator__n_estimators': 500}


In [29]:
grid_search_improved.score(X_test, Y_test)

0.2956972841013122

In [32]:
grid_search_improved.get_params()

{'cv': 3,
 'error_score': nan,
 'estimator__memory': None,
 'estimator__steps': [('features',
   FeatureUnion(transformer_list=[('text_pipeline',
                                   Pipeline(steps=[('vect',
                                                    CountVectorizer(tokenizer=<function tokenize_improved at 0x7f9b6bb0f0a0>)),
                                                   ('tfidf',
                                                    TfidfTransformer())])),
                                  ('isverb', StartingVerbExtractor())])),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                          n_jobs=-1)))],
 'estimator__verbose': False,
 'estimator__features': FeatureUnion(transformer_list=[('text_pipeline',
                                 Pipeline(steps=[('vect',
                                                  CountVectorizer(tokenizer=<function tokenize_improved at 0x7f9b6bb0f0a0>)),
  

### 9. Export your model as a pickle file

In [31]:
with open('model_improved.pickle', 'wb') as file:
    pickle.dump(grid_search_improved, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
# Y_pred = pipeline.predict(X_test)

# list_precision, list_recall, list_f1 = [], [], []

# # Iterate 36 target columns and generate a classification report for each
# for i, col in enumerate(df.columns[4:]):
    
#     rslt = classification_report(Y_test[:, i], Y_pred[:, i])
    
#     # weighted avg scores are in the 2nd last line: 
#     score_line = rslt.split('\n')[-2]
#     score_line_split = score_line.split()
    
#     # scores are in the 2nd to 4th places of splitted texts of score_line
#     precision_score = float (score_line_split[2])
#     list_precision.append(precision_score)
    
#     recall_score = float (score_line_split[3])
#     list_recall.append(recall_score)
    
#     f1_score = float (score_line_split[4])
#     list_f1.append(f1_score)
    
#     print(f'{i} Target column {col}:')
#     print(rslt)
#     print()
# # mean of weighted avg precision, recall and f1 
# print ('mean of weighted avg precision: {:.2f}'.format(sum(list_precision)/len(list_precision)))
# print ('mean of weighted avg recall: {:.2f}'.format(sum(list_recall)/len(list_recall)))
# print ('mean of weighted avg f1: {:.2f}'.format(sum(list_f1)/len(list_f1)))

In [ ]:
# # import libraries
# import re
# import numpy as np
# import pandas as pd

# import nltk
# nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# import string

# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix, f1_score, classification_report

# # from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.base import BaseEstimator, TransformerMixin

# from sqlalchemy import create_engine